In [1]:
#何も考えずに全部のデータを結合してモデルに放り込む

In [11]:
import pandas as pd
import numpy as np
import glob 
from datetime import timedelta
from lightgbm import Dataset, LGBMRegressor
from matplotlib import pyplot as plt
import lightgbm as lgb
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils import all_estimators

from matplotlib import pyplot as plt


In [13]:
station=pd.read_csv("dataset/station.csv")
status=pd.read_csv("dataset/status.csv")
trip=pd.read_csv("dataset/trip.csv")
weather=pd.read_csv("dataset/weather.csv")
submit=pd.read_csv('dataset/sample_submit.csv',names=(0,1))

In [16]:
status["date"] = pd.to_datetime(status[["year", "month", "day", "hour"]])
status["date_day"] = pd.to_datetime(status[["year", "month", "day"]])

weather['date'] = pd.to_datetime(weather['date'])

station['installation_date'] = pd.to_datetime(station['installation_date'])
status['week_num'] = status['date'].dt.weekday


In [28]:
time_lis=list(range(1,24))
station_lis=list(range(0,70))
day_lis=list(status[(status['predict']==1)&(status['station_id']==0)]['date_day'].unique())


In [19]:
status.head()

,id,year,month,day,hour,station_id,bikes_available,predict,date,date_day,week_num,bikes_available_at0
0,0,2013,9,1,0,0,11.0,0,2013-09-01 00:00:00,2013-09-01,6,11.0
1,1,2013,9,1,1,0,11.0,0,2013-09-01 01:00:00,2013-09-01,6,11.0
2,2,2013,9,1,2,0,11.0,0,2013-09-01 02:00:00,2013-09-01,6,11.0
3,3,2013,9,1,3,0,11.0,0,2013-09-01 03:00:00,2013-09-01,6,11.0
4,4,2013,9,1,4,0,11.0,0,2013-09-01 04:00:00,2013-09-01,6,11.0


In [17]:
t = status.groupby(['station_id', 'date']).first()['bikes_available'].reset_index()
#24回リピートすることでデータのサイズを合わせる
t = pd.DataFrame(np.repeat(t.values, 24, axis=0))
t.columns = ['station_id', 'date', 'bikes_available_at0']
status['bikes_available_at0'] = t['bikes_available_at0']

In [20]:
t

,station_id,date,bikes_available_at0
0,0,2013-09-01 00:00:00,11.0
1,0,2013-09-01 00:00:00,11.0
2,0,2013-09-01 00:00:00,11.0
3,0,2013-09-01 00:00:00,11.0
4,0,2013-09-01 00:00:00,11.0
...,...,...,...
29433595,69,2015-08-31 23:00:00,8.0
29433596,69,2015-08-31 23:00:00,8.0
29433597,69,2015-08-31 23:00:00,8.0
29433598,69,2015-08-31 23:00:00,8.0


In [18]:
cols = ['city']

encoder = ce.CountEncoder()
temp_ = encoder.fit_transform(station[cols]).add_prefix("CE_")

station = pd.concat([station, temp_], axis=1)

cols = ['events']

encoder = ce.CountEncoder()
temp_ = encoder.fit_transform(weather[cols]).add_prefix("CE_")

weather = pd.concat([weather, temp_], axis=1)


In [21]:
station

,station_id,lat,long,dock_count,city,installation_date,CE_city
0,0,37.32973,-121.90178,27,city1,2013-08-06,16
1,1,37.33070,-121.88898,15,city1,2013-08-05,16
2,2,37.33399,-121.89490,11,city1,2013-08-06,16
3,3,37.33141,-121.89320,19,city1,2013-08-05,16
4,4,37.33672,-121.89407,15,city1,2013-08-07,16
...,...,...,...,...,...,...,...
65,65,37.78963,-122.40081,27,city2,2013-08-25,35
66,66,37.35260,-121.90573,15,city1,2013-12-31,16
67,67,37.79854,-122.40086,15,city2,2014-01-22,35
68,68,37.49127,-122.23623,15,city3,2014-02-20,7


In [22]:
status_station=pd.merge(status,station,how="left")
all_df=pd.merge(status_station,weather,left_on='date_day',right_on='date')



In [23]:
#ステーション設置からの経過日数
all_df['inst_days']=(all_df['date_y']-all_df['installation_date'])/timedelta(days=1)


In [24]:
test_col=['hour', 'station_id','week_num','bikes_available_at0','lat', 'long', 'dock_count', 'CE_city','max_temperature', 'mean_temperature', 'min_temperature',
 'max_dew_point', 'mean_dew_point', 'min_dew_point', 'max_humidity', 'mean_humidity', 'min_humidity', 'max_sea_level_pressure','mean_sea_level_pressure', 'min_sea_level_pressure', 'max_visibility',
 'mean_visibility', 'min_visibility', 'max_wind_Speed','mean_wind_speed', 'precipitation', 'cloud_cover','wind_dir_degrees', 'CE_events', 'inst_days'
]

In [29]:
train=all_df[all_df['date_x']<=day_lis[0]]
pres=all_df[all_df['predict']==1]

In [36]:
train

,id,year,month,day,hour,station_id,bikes_available,predict,date_x,date_day,...,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,events,wind_dir_degrees,CE_events,inst_days
0,0,2013,9,1,0,0,11.0,0,2013-09-01 00:00:00,2013-09-01,...,10,10,14,4,0.0,1,NaN,354,636,26.0
1,1,2013,9,1,1,0,11.0,0,2013-09-01 01:00:00,2013-09-01,...,10,10,14,4,0.0,1,NaN,354,636,26.0
2,2,2013,9,1,2,0,11.0,0,2013-09-01 02:00:00,2013-09-01,...,10,10,14,4,0.0,1,NaN,354,636,26.0
3,3,2013,9,1,3,0,11.0,0,2013-09-01 03:00:00,2013-09-01,...,10,10,14,4,0.0,1,NaN,354,636,26.0
4,4,2013,9,1,4,0,11.0,0,2013-09-01 04:00:00,2013-09-01,...,10,10,14,4,0.0,1,NaN,354,636,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614760,1147560,2014,9,1,0,65,10.0,0,2014-09-01 00:00:00,2014-09-01,...,10,10,17,5,0.0,0,NaN,342,636,372.0
614784,1165080,2014,9,1,0,66,9.0,0,2014-09-01 00:00:00,2014-09-01,...,10,10,17,5,0.0,0,NaN,342,636,244.0
614808,1182600,2014,9,1,0,67,7.0,0,2014-09-01 00:00:00,2014-09-01,...,10,10,17,5,0.0,0,NaN,342,636,222.0
614832,1200120,2014,9,1,0,68,8.0,0,2014-09-01 00:00:00,2014-09-01,...,10,10,17,5,0.0,0,NaN,342,636,193.0


In [31]:
from sklearn import ensemble
regr = RandomForestRegressor(max_depth=15, random_state=0)
regr.fit(train_X, train_y)
val_predict = regr.predict(valid_X)

ImportError: cannot import name 'available_if' from 'sklearn.utils.metaestimators' (/Users/chibaren/Library/Python/3.8/lib/python/site-packages/sklearn/utils/metaestimators.py)

In [ ]:
submit[1]=list(val_predict)

In [27]:
submit.to_csv("submission.csv",index=False, header=False)